In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [44]:
nt = 50 # number of temperature points
N=10  # size of lattice, N x N
eqsteps = 2000
mcsteps = 2500

T = np.linspace(1.53, 3.28, nt)
Energy, Magnetization, SpecificHeat, Susceptibility = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)


In [47]:
a = np.random.randint(0, N)
b = np.random.randint(0, N)
s =  state[a, b]
print(a,b)
print((a+1)%N,b)
print((a-1)%N,b)
print(a,(b+1)%N)
print(a,(b-1)%N)
print(s)

7 8
8 8
6 8
7 9
7 7
1


In [6]:
state = 2*np.random.randint(2,size=(N,N))-1
print(state) 

[[-1  1 -1 -1 -1  1  1  1  1  1]
 [ 1  1  1 -1  1 -1 -1  1 -1  1]
 [ 1 -1 -1  1 -1  1  1  1 -1 -1]
 [ 1 -1  1  1 -1 -1  1 -1  1  1]
 [ 1  1  1  1 -1  1  1  1 -1  1]
 [ 1 -1 -1  1 -1 -1  1  1  1 -1]
 [ 1  1  1 -1  1 -1 -1 -1 -1  1]
 [-1 -1  1 -1 -1 -1  1 -1  1 -1]
 [ 1 -1 -1 -1 -1  1  1 -1  1 -1]
 [ 1 -1 -1  1 -1  1  1 -1 -1 -1]]
